
Project Objective:

Detect fraudulent credit card transactions using machine learning.
Compare Logistic Regression and XGBoost models.
Dataset:

Source: Kaggle (Credit Card Fraud Detection Dataset)
Features: Transaction features (V1, V2, ..., Amount)
Target: Class (0 = non-fraud, 1 = fraud)

In [32]:
! pip install missingno
! pip install smote-variants

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/155.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/155.5 MB 4.8 MB/s eta 0:00:33
    --------------------------------------- 2.1/155.5 MB 6.5 MB/s eta 0:00:24
   - -------------------------------------- 3.9/155.5 MB 6.9 MB/s eta 0:00:22
   - ------------------------------

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
import kagglehub
import os
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [51]:
import tensorflow as tf # deep learning frame work
from tensorflow import keras #high level api for building models
from tensorflow.keras import layers #neural networklayers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf  # Deep learning framework
from tensorflow import keras  # High-level API for building models
from tensorflow.keras import layers  # Neural network layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.model_selection import train_test_split #split data into test train sets
from sklearn.preprocessing import StandardScaler #normalize features
from sklearn.svm import LinearSVC #using LinearSCC for effeciency on large data set
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, f_classif # Import feature selection tools
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression #LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, accuracy_score, f1_score  # Evaluation metrics

pd.set_option("display.max_columns", None) #to displaythe total number od cloumns psent in the data set

In [52]:

path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\mlg-ulb\creditcardfraud\versions\3


In [54]:

os.listdir(r"C:\Users\DELL\.cache\kagglehub\datasets\mlg-ulb\creditcardfraud\versions\3")

['creditcard.csv']

In [55]:
Data=pd.read_csv(os.path.join(path, "creditcard.csv"))

In [67]:
pd.set_option('display.max_columns', None) # to display the total number columns present in the dataset

In [68]:
Data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [58]:
Data.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [59]:
Data.duplicated().sum()

np.int64(1081)

After checking we see we have Duplicated in our Datasets, so, we will go ahead and remove them.

In [61]:
#Droping duplicates
Data.drop_duplicates(inplace=True)

checking Duplicates after removing.

In [62]:
Data.duplicated().sum()

np.int64(0)

In [66]:
# Display columns names
Data.columns.to_list() 

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount',
 'Class']

In [72]:
#Get summr tastitics for each columns and transpose for better readibility
Data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Time,283726.0,94811.077600,47481.047891,0.000000,54204.750000,84692.500000,139298.000000,172792.000000
V1,283726.0,0.005917,1.948026,-56.407510,-0.915951,0.020384,1.316068,2.454930
V2,283726.0,-0.004135,1.646703,-72.715728,-0.600321,0.063949,0.800283,22.057729
V3,283726.0,0.001613,1.508682,-48.325589,-0.889682,0.179963,1.026960,9.382558
V4,283726.0,-0.002966,1.414184,-5.683171,-0.850134,-0.022248,0.739647,16.875344
V5,283726.0,0.001828,1.377008,-113.743307,-0.689830,-0.053468,0.612218,34.801666
V6,283726.0,-0.001139,1.331931,-26.160506,-0.769031,-0.275168,0.396792,73.301626
V7,283726.0,0.001801,1.227664,-43.557242,-0.552509,0.040859,0.570474,120.589494
V8,283726.0,-0.000854,1.179054,-73.216718,-0.208828,0.021898,0.325704,20.007208
V9,283726.0,-0.001596,1.095492,-13.434066,-0.644221,-0.052596,0.595977,15.594995


In [74]:
Data.shape

(283726, 31)

In [78]:
#count the numerical columns
num_cols = Data.select_dtypes(include=["number"]).shape[1]

#count categorical columns (object, catergory types)
cat_cols = Data.select_dtypes(include=["object","category"]).shape[1]

print(f"Numerical columns: {num_cols}")
print(f"Categorical columns: {cat_cols}")

Numerical columns: 31
Categorical columns: 0
